#################################################
##      뉴스기사 키워드 추출 및 테마 추천       ##
#################################################

0. 라이브러리 임포트

In [124]:
# -*- coding: utf-8 -*-
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import pickle
import re
import itertools

1. 뉴스기사 선언

In [125]:
article = """
[서울=뉴시스]송연주 기자 = 코로나19 환자가 급속도로 증가하면서 코로나19 치료제 및 진단키트를 구하고 싶어도 못 구하는 상황에 이르렀다. 코로나 유행 때마다 품절 대란이 따랐던 감기약, 해열제의 수요도 증가하며 제약회사는 생산공장 풀가동에 나섰다.

18일 관련업계에 따르면 대원제약, 유한양행은 코로나19 재유행 조짐을 보이자 최근 감기약 생산공장 전체가동에 돌입했다.



대원제약의 감기약 브랜드 '콜대원', 진해거담제 '코대원', 유한양행 진해거담제 '코푸시럽' 등은 지난 유행 당시 품절이 끊이지 않았던 의약품이다.

현재 전반적인 감기약 수급 불안이 나타나는 건 아니지만 공급 부족을 우려한 일부 약국의 사재기 현상도 있다는 게 업계의 설명이다.

제약업계 관계자는 "최근 들어 감기약 수요가 급증해 풀가동에 들어갔다"며 "전국적으로 감기약, 해열제가 부족한 건 아니지만 일부 약국에선 물량 부족 현상이 있고 불안감으로 인한 사재기 심리도 발동하고 있어 3교대 풀가동 중"이라고 말했다.



질병관리청에 최근 4주간 코로나19 입원환자는 7월 셋째 주 226명에서, 7월 넷째 주 472명, 8월 첫째 주 869명, 8월 둘째 주 1357명으로 매주 2배 가까이 증가하는 중이다. 8월 2주차 1357명은 올해 들어 가장 높은 수준이다.

65세 이상이 전체 입원환자 수의 65.4%로 가장 많았고, 50~64세가 18.2%로 뒤를 이었다.

이번 유행은 오미크론 세부계통인 KP.3 변이 바이러스가 주도하고 있다. KP.3 변이 점유율은 지난달 기준 45.5%로 6월보다 33.4%포인트(P) 늘었다. KP.3는 전 세계적으로 증가하는 추세를 보여 세계보건기구(HWO)도 모니터링변이로 감시 중이다.

팍스로비드, 라게브리오 같은 코로나19 치료제 공급은 상당히 부족한 상황이다. 복지위원회 한지아 국민의힘 의원이 질병관리청으로부터 받은 자료에 따르면 8월 첫주 전국 약국·의료기관의 코로나 치료제 신청량은 19만8000명분이었지만 공급량은 3만3000명분에 불과했다. 신청량의 16.7% 수준이다. 이들 약은 고령자, 면역저하자 등 고위험군에 권장된다.

감기약 수요도 함께 늘면서 홍정익 질병관리청 감염병정책국장은 "증상이 유사하다면 코로나19 치료제가 아닌 감기약을 먹어도 된다"며 "대부분 젊은층들은 일반 호흡기 감염병처럼 해열제 등으로 조절하면 된다"고 언급하기도 했다.

서울 양천구 소재 약사는 "감기약·해열제의 수급 불안 우려는 공급 부족이 아니라 일부 약국의 사재기로 인한 것이다. 작년 같은 품절 사태는 없을 것"이라고 말했다.

이어 "하지만 진단키트와 치료제는 상황이 다르다"며 "진단키트는 1개(2개 들이) 2000원이었는데 4000원으로 오르더니 지금은 4800원에 주문해도 못 받는 상태다. 정말 제품이 없는 게 아니라, 코로나19 초기 때 마스크처럼 중간 유통업자가 가격이 더 오를 때까지 제품을 내놓지 않는 것으로 보여 우려스럽다"고 말했다.
"""

2. 텍스트 전처리

In [126]:
article = article.replace('\n', '')
split_article = re.split(r'(\s*[a-zA-Z]+(?:[^\w\s]+[a-zA-Z]+)*\s*)', article)       #   한글과 영어 나누기
split_article = [part.strip() for part in split_article if part.strip()]            #   불필요한 공백 지우기

okt = Okt()                         #   형태소 분석기 Okt
nouns=[]
for sentence in split_article:
    sen = okt.nouns(sentence)
    if len(sen) != 0:               #   명사를 리스트에 추가
        nouns.extend(sen)
    else:
        nouns.append(sentence)      #   영어도 리스트에 추가(형태소 분석 후 길이가 0이면 영어로 판단)

text = ' '.join(nouns)              #   리스트를 하나의 문자열로

print(f'전처리 후 뉴스 기사 : {text}')

전처리 후 뉴스 기사 : 서울 뉴시스 송 연주 기자 코로나 환자 급속도 증가 코로나 치료 제 및 진단 키트 구 못 구 상황 이르렀다 코로나 유행 때 품절 대란 감기 약 해열제 수요 증가 제약회사 생산 공장 풀 가동 관련 업계 대원 제약 유한양행 코로나 재 유행 조짐 보이 최근 감기 약 생산 공장 전체 가동 돌입 제약 감기 약 브랜드 콜 대원 제 코 대원 유한양행 제 코 시럽 등 지난 유행 당시 품절 의약품 현재 전반 감기 약 수급 불안 건 공급 부족 우려 일부 약국 사재기 현상 게 업계 설명 제약업 관계자 최근 감기 약 수요 급증 풀 가동 며 전국 감기 약 해열제 건 일부 약국 물량 부족 현상 불안감 사재기 심리 발동 교대 풀 가동 중 말 질병 관리 청 최근 주간 코로나 입원 환자 주 명 넷째 주 명 첫째 주 명 둘째 주 명 배 가까이 증가 중이 주차 명 올해 가장 수준 세 이상 전체 입원 환자 수의 로 가장 세 로 뒤 이번 유행 오미크론 세부 계통 KP 변 바이러스 KP 변 점유 율 지난달 기준 로 포인트 P ) 늘었다. KP 전 세계 증가 추세 세계보건기구 HWO 도 모니터링 변 감시 중이 팍스 비드 게브 리오 코로나 치료 제 공급 상황 복지 위원회 한지 국민 힘 의원 질병 관리 청 자료 주 전국 약국 의료 기관 코로나 치료 제 신청량 명분 급량 명분 신청량 수준 이 약 고령자 면역 하자 등 위험 권장 감기 약 수요 홍 정익 질병 관리 청 감염병 정책 국장 증상 코로나 치료 제 감기 약 며 대부분 층 일반 호흡기 감염병 해열제 등 조절 고 언급 서울 양천구 소재 약사 감기 약 해열제 수급 불안 우려 공급 부족 일부 약국 사재기 것 작년 품절 사태 것 말 진단 키트 치료 제 상황 며 진단 키트 개 개 지금 주문 못 상태 정말 제품 게 코로나 초기 때 마스크 중간 유통업 가격 더 때 제품 것 우려 고 말


3. N-Gram 생성 및 SBERT

In [127]:
n_gram_range = (1,2)
word_vectorize = CountVectorizer(ngram_range=n_gram_range).fit([text])   #   1 ~ 2 글자로 N-Gram 생성
n_gram_words = word_vectorize.get_feature_names_out()
print(f'N-Gram 개수 : {len(n_gram_words)}')
print(f'N-Gram 다섯개 출력 : {n_gram_words[:5]}')

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')  #   Sentence Transformers
article_embedding = model.encode([article])
n_gram_embeddings = model.encode(n_gram_words)
print(f'SBERT 분석 후 뉴스 기사 : {article_embedding}')
print(f'SBERT 분석 후 N-Gram : {n_gram_embeddings}')

N-Gram 개수 : 338
N-Gram 다섯개 출력 : ['hwo' 'hwo 모니터링' 'kp' 'kp 바이러스' 'kp 세계']
SBERT 분석 후 뉴스 기사 : [[-1.16439030e-01 -1.09249443e-01  1.06583619e+00 -5.49926013e-02
   1.46645769e-01 -7.52584100e-01  4.77469981e-01  7.69079253e-02
   2.51852244e-01 -3.40351939e-01 -2.51321971e-01 -6.37713373e-02
   5.81892319e-02  3.91669869e-01  1.69146098e-02  7.27439076e-02
  -2.90680975e-01 -7.17419758e-02  2.61696205e-02  1.57818913e-01
  -3.11367989e-01 -6.77919745e-01  1.25799680e+00  5.18618822e-01
   5.35489619e-01  1.74542721e-02  1.37457903e-02  4.23052609e-01
   4.87066582e-02  5.30523658e-01 -2.31470957e-01 -7.23368943e-01
  -1.78606823e-01 -6.24754727e-01 -1.28643677e-01  8.31649780e-01
  -5.03143072e-01  1.26988447e+00  4.04207557e-02 -3.34092677e-01
  -9.87871885e-01  1.23037118e-02  9.38994586e-02 -3.34923327e-01
  -3.36467683e-01 -1.85703561e-02 -6.20917082e-01 -3.20871294e-01
   3.32289815e-01  1.10490620e-01  5.70190191e-01 -2.53732130e-02
   3.59003365e-01 -5.47347367e-01 -1.19626187e-01

4. Max Sum Similarity

In [128]:
def max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n, variety):
    #   뉴스 기사와 N-Gram의 유사도
    distances = cosine_similarity(article_embedding, n_gram_embeddings)
    #   N-Gram들 사이의 유사도
    distances_candidates = cosine_similarity(n_gram_embeddings, n_gram_embeddings)

    #   뉴스 기사와 유사도가 높은 N-Gram을 variety개 가져오기
    words_idx = list(distances.argsort()[0][-variety:])
    words_vals = [n_gram_words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    #   그 중 유사도가 가장 낮은 조합 찾기
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

news_keywords = max_sum_sim(article_embedding, n_gram_embeddings, n_gram_words, top_n=5, variety=10)
print(f'뉴스 키워드 5개 : {news_keywords}')

뉴스 키워드 5개 : ['약국 사재기', '감염병 해열제', '해열제 수요', '전국 약국', '증가 제약회사']


5. 테마 가져오기

In [129]:
#       딕셔너리에서 key 값 가져오기
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

keys = loaded_dict.keys()
keys_list = list(keys)

new_keys_list = [re.sub(r'[^a-zA-Z0-9가-힣\s]', ' ', key) for key in keys_list]     #   테마 전처리
key_embeddings = model.encode(new_keys_list)                                        #   테마  SBERT 분석

6. 코사인 거리 기반 & Max Sum Similarity 테마 추천

In [130]:
top_n = 5
distances = cosine_similarity(article_embedding, key_embeddings)
cosine_recommend = [keys_list[index] for index in distances.argsort()[0][-top_n:]]

# maxsum_recommend = max_sum_sim(article_embedding, key_embeddings, keys_list, top_n=3, variety=30)

article = article.replace('.', '.\n')
print(f'뉴스 기사 : \n{article}')
print(f'뉴스 키워드 5개 : {news_keywords}')

print(f'\n뉴스 기사와 연관 되어 있는 테마 5개')
for i in range(0,5):
    print(f'{i+1}. {cosine_recommend[-i]} : {loaded_dict[cosine_recommend[-i]]}')

# print(f'\n뉴스 기사와 연관될 수 있는 테마 3개')
# maxsum_index = 1
# for maxsum in maxsum_recommend:
#     print(f'{maxsum_index}. {maxsum} : {loaded_dict[maxsum]}')
#     maxsum_index += 1

뉴스 기사 : 
[서울=뉴시스]송연주 기자 = 코로나19 환자가 급속도로 증가하면서 코로나19 치료제 및 진단키트를 구하고 싶어도 못 구하는 상황에 이르렀다.
 코로나 유행 때마다 품절 대란이 따랐던 감기약, 해열제의 수요도 증가하며 제약회사는 생산공장 풀가동에 나섰다.
18일 관련업계에 따르면 대원제약, 유한양행은 코로나19 재유행 조짐을 보이자 최근 감기약 생산공장 전체가동에 돌입했다.
대원제약의 감기약 브랜드 '콜대원', 진해거담제 '코대원', 유한양행 진해거담제 '코푸시럽' 등은 지난 유행 당시 품절이 끊이지 않았던 의약품이다.
현재 전반적인 감기약 수급 불안이 나타나는 건 아니지만 공급 부족을 우려한 일부 약국의 사재기 현상도 있다는 게 업계의 설명이다.
제약업계 관계자는 "최근 들어 감기약 수요가 급증해 풀가동에 들어갔다"며 "전국적으로 감기약, 해열제가 부족한 건 아니지만 일부 약국에선 물량 부족 현상이 있고 불안감으로 인한 사재기 심리도 발동하고 있어 3교대 풀가동 중"이라고 말했다.
질병관리청에 최근 4주간 코로나19 입원환자는 7월 셋째 주 226명에서, 7월 넷째 주 472명, 8월 첫째 주 869명, 8월 둘째 주 1357명으로 매주 2배 가까이 증가하는 중이다.
 8월 2주차 1357명은 올해 들어 가장 높은 수준이다.
65세 이상이 전체 입원환자 수의 65.
4%로 가장 많았고, 50~64세가 18.
2%로 뒤를 이었다.
이번 유행은 오미크론 세부계통인 KP.
3 변이 바이러스가 주도하고 있다.
 KP.
3 변이 점유율은 지난달 기준 45.
5%로 6월보다 33.
4%포인트(P) 늘었다.
 KP.
3는 전 세계적으로 증가하는 추세를 보여 세계보건기구(HWO)도 모니터링변이로 감시 중이다.
팍스로비드, 라게브리오 같은 코로나19 치료제 공급은 상당히 부족한 상황이다.
 복지위원회 한지아 국민의힘 의원이 질병관리청으로부터 받은 자료에 따르면 8월 첫주 전국 약국·의료기관의 코로나 치료제 신청량은 19만8000명분이었지만 공급량은 3만300